In [ ]:
#pip install yt-dlp

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import yt_dlp
import time
import os
from urllib.parse import urlparse

# wget 사용으로 메시지 출력
def check_wget_installed():
    if os.system("wget --version") != 0:
        print("[ERROR] wget이 설치되어 있지 않습니다. 설치 후 다시 시도해주세요.")
        exit(1)

# 형식 목록 확인
def list_formats(video_url):
    try:
        with yt_dlp.YoutubeDL({'listformats': True}) as ydl:
            info = ydl.extract_info(video_url, download=False)
            print("\n[INFO] 사용 가능한 형식:")
            for fmt in info['formats']:
                print(f"ID: {fmt['format_id']}, Resolution: {fmt['resolution']}, Size: {fmt.get('filesize', 'Unknown')}")
    except Exception as e:
        print(f"[ERROR] 형식 목록을 가져오는 중 오류 발생: {e}")

# 자막만 다운로드 함수
def download_subtitles(video_url, output_filename):
    ydl_opts = {
        'skip_download': True,  # 동영상 다운로드 건너뜀
        'writesubtitles': True,  # 자막 다운로드 활성화
        'writeautomaticsub': True,  # 자동 생성 자막 다운로드 활성화
        'subtitleslangs': ['en', 'ko'],  # 다운로드할 자막 언어 (영어와 한국어 우선)
        'subtitlesformat': 'vtt',  # 자막 파일 형식 (VTT)
        'outtmpl': f'{output_filename}.%(ext)s',  # 파일 이름 템플릿
        'quiet': False,  # 메시지를 출력
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=False)
            available_subs = info.get('subtitles', {})
            if not available_subs:
                print(f"[WARNING] {video_url}에는 자막이 없습니다.")
                return False

            # 영어와 한국어가 없으면 사용 가능한 첫 번째 언어 선택
            if 'en' not in available_subs and 'ko' not in available_subs:
                first_lang = next(iter(available_subs.keys()))
                ydl_opts['subtitleslangs'] = [first_lang]
                print(f"[INFO] 영어와 한국어 자막이 없어 {first_lang} 자막을 다운로드합니다.")

            ydl.download([video_url])
            print(f"[INFO] 자막 다운로드 완료: {output_filename}")

        # 자막에서 시간 및 기타 정보 제거하고 텍스트만 저장
        for lang in ydl_opts['subtitleslangs']:
            subtitle_file = f"{output_filename}.{lang}.vtt"
            if os.path.exists(subtitle_file):
                with open(subtitle_file, 'r', encoding='utf-8') as file:
                    lines = file.readlines()
                text_lines = [
                    line.strip() for line in lines
                    if not ("-->") in line and line.strip() \
                    and not line.startswith("WEBVTT") and not line.startswith("NOTE") \
                    and not line.startswith("STYLE") and not line.startswith("HEADER") \
                    and not line.startswith("X-TIMESTAMP-MAP") \
                    and not line.startswith("(")  # 배경 음악 또는 효과 제거
                ]

                # 순서 번호를 줄 번호와 공백으로 시작하는 라인 필터링
                cleaned_lines = []
                for line in text_lines:
                    if not line.split()[0].isdigit():
                        cleaned_lines.append(line)

                text_output_file = f"{output_filename}_{lang}_text_only.txt"
                with open(text_output_file, 'w', encoding='utf-8') as text_file:
                    text_file.write("\n".join(cleaned_lines))
                print(f"[INFO] 텍스트 자막 파일 저장 완료: {text_output_file}")
    except Exception as e:
        print(f"[ERROR] 자막 다운로드 실패: {video_url}, 오류: {e}")
        return False

    return True

# 자막 파일 저장 경로 확인 및 출력
def print_subtitle_location(output_filename):
    subtitle_files = [
        f"{output_filename}.en.vtt",
        f"{output_filename}.ko.vtt",
        f"{output_filename}_en_text_only.txt",
        f"{output_filename}_ko_text_only.txt"
    ]
    for file in subtitle_files:
        if os.path.exists(file):
            print(f"[INFO] 파일 저장 위치: {os.path.abspath(file)}")
        else:
            print(f"[WARNING] 파일을 찾을 수 없습니다: {file}")

# 병렬 자막 다운로드 함수
def download_subtitles_concurrently(video_urls):
    results = []
    with ThreadPoolExecutor(max_workers=13) as executor:  # 최대 13개의 작업 동시 실행
        futures = []
        for idx, video_url in enumerate(video_urls):
            parsed_url = urlparse(video_url)
            output_filename = parsed_url.path.strip("/").replace("/", "_") or f"Subtitle_{idx + 1}"
            futures.append(executor.submit(download_subtitles, video_url, output_filename))
        
        for idx, future in enumerate(futures):
            result = future.result()
            results.append((video_urls[idx], result))
    return results

# 메인 실행
if __name__ == "__main__":
    # wget 설치 여부 확인
    check_wget_installed()

    # 다운로드할 동영상 URL 목록
    video_urls = [
        "https://www.ces.tech/videos/2025/january/tech-and-the-insightful-life/",
        "https://www.ces.tech/videos/2025/january/the-future-of-immersive-tv/",
        "https://www.ces.tech/videos/2025/january/how-ai-is-changing-consumer-behavior/",
        "https://www.ces.tech/videos/2025/january/ai-in-hollywood-the-tech-behind-the-hype/",
        "https://www.ces.tech/videos/2025/january/ai-in-hollywood-applied-ai-the-good-the-bad-and-the-unknown/",
        "https://www.ces.tech/videos/2025/january/safe-money-venture-leaders-leading-the-way-in-cybersecurity/",
        "https://www.ces.tech/videos/2025/january/securing-the-iot-what-is-the-lay-of-the-land/",
        "https://www.ces.tech/videos/2025/january/us-cyber-trust-mark-getting-ready-for-certification/",
        "https://www.ces.tech/videos/2025/january/the-cybersecurity-talent-gap/",
        "https://www.ces.tech/videos/2025/january/do-you-need-a-chief-cybersecurity-officer/",
        "https://www.ces.tech/videos/2025/january/data-collection-privacy-and-why-you-should-care/",
        "https://www.ces.tech/videos/2025/january/data-breach-protection-and-prevention/",
        # 필요한 다른 URL 추가
    ]

    # 형식 목록 확인
    for url in video_urls:
        list_formats(url)

    # 자막만 다운로드 실행
    start_time = time.time()
    results = download_subtitles_concurrently(video_urls)

    # 결과 요약 출력
    print("\n[INFO] 자막 다운로드 결과 요약:")
    for idx, (url, success) in enumerate(results):
        status = "성공" if success else "실패"
        print(f"  {idx + 1}. {url} -> {status}")

        if success:
            output_filename = urlparse(url).path.strip("/").replace("/", "_") or f"Subtitle_{idx + 1}"
            print_subtitle_location(output_filename)

    print(f"\n[INFO] 모든 자막 다운로드 완료! 소요 시간: {time.time() - start_time:.2f}초")


[ERROR] wget이 설치되어 있지 않습니다. 설치 후 다시 시도해주세요.
[generic] Extracting URL: https://www.ces.tech/videos/2025/january/tech-and-the-insightful-life/
[generic] tech-and-the-insightful-life: Downloading webpage


[generic] tech-and-the-insightful-life: Extracting information
[brightcove:new] Extracting URL: https://players.brightcove.net/5629323166001/G9w6yOa2YM_default/index.html?videoId=6366881659112#...ghtful-life%2F%22%7D
[brightcove:new] 6366881659112: Downloading JSON metadata
[brightcove:new] 6366881659112: Downloading m3u8 information
[brightcove:new] 6366881659112: Downloading m3u8 information
[brightcove:new] 6366881659112: Downloading m3u8 information
[brightcove:new] 6366881659112: Downloading m3u8 information
[brightcove:new] 6366881659112: Downloading MPD manifest
[brightcove:new] 6366881659112: Downloading MPD manifest
[brightcove:new] 6366881659112: Downloading MPD manifest
[brightcove:new] 6366881659112: Downloading MPD manifest
[info] Available formats for 6366881659112:
ID                                          EXT RESOLUTION |   FILESIZE   TBR PROTO | VCODEC        VBR ACODEC      ABR ASR MORE INFO
---------------------------------------------------------------------------

[generic] the-future-of-immersive-tv: Extracting information
[brightcove:new] Extracting URL: https://players.brightcove.net/5629323166001/G9w6yOa2YM_default/index.html?videoId=6366880559112#...mmersive-tv%2F%22%7D
[brightcove:new] 6366880559112: Downloading JSON metadata
[brightcove:new] 6366880559112: Downloading m3u8 information
[brightcove:new] 6366880559112: Downloading m3u8 information
[brightcove:new] 6366880559112: Downloading m3u8 information
[brightcove:new] 6366880559112: Downloading m3u8 information
[brightcove:new] 6366880559112: Downloading MPD manifest
[brightcove:new] 6366880559112: Downloading MPD manifest
[brightcove:new] 6366880559112: Downloading MPD manifest
[brightcove:new] 6366880559112: Downloading MPD manifest
[info] Available formats for 6366880559112:
ID                                          EXT RESOLUTION |    FILESIZE   TBR PROTO | VCODEC        VBR ACODEC      ABR ASR MORE INFO
----------------------------------------------------------------------------

[generic] how-ai-is-changing-consumer-behavior: Extracting information
[brightcove:new] Extracting URL: https://players.brightcove.net/5629323166001/G9w6yOa2YM_default/index.html?videoId=6366885573112#...er-behavior%2F%22%7D
[brightcove:new] 6366885573112: Downloading JSON metadata
[brightcove:new] 6366885573112: Downloading m3u8 information
[brightcove:new] 6366885573112: Downloading m3u8 information
[brightcove:new] 6366885573112: Downloading m3u8 information
[brightcove:new] 6366885573112: Downloading m3u8 information
[brightcove:new] 6366885573112: Downloading MPD manifest
[brightcove:new] 6366885573112: Downloading MPD manifest
[brightcove:new] 6366885573112: Downloading MPD manifest
[brightcove:new] 6366885573112: Downloading MPD manifest
[info] Available formats for 6366885573112:
ID                                          EXT RESOLUTION |   FILESIZE   TBR PROTO | VCODEC        VBR ACODEC      ABR ASR MORE INFO
-------------------------------------------------------------------

[generic] ai-in-hollywood-the-tech-behind-the-hype: Extracting information
[brightcove:new] Extracting URL: https://players.brightcove.net/5629323166001/G9w6yOa2YM_default/index.html?videoId=6366887292112#...nd-the-hype%2F%22%7D
[brightcove:new] 6366887292112: Downloading JSON metadata
[brightcove:new] 6366887292112: Downloading m3u8 information
[brightcove:new] 6366887292112: Downloading m3u8 information
[brightcove:new] 6366887292112: Downloading m3u8 information
[brightcove:new] 6366887292112: Downloading m3u8 information
[brightcove:new] 6366887292112: Downloading MPD manifest
[brightcove:new] 6366887292112: Downloading MPD manifest
[brightcove:new] 6366887292112: Downloading MPD manifest
[brightcove:new] 6366887292112: Downloading MPD manifest
[info] Available formats for 6366887292112:
ID                                          EXT RESOLUTION |   FILESIZE   TBR PROTO | VCODEC        VBR ACODEC      ABR ASR MORE INFO
---------------------------------------------------------------

...

[brightcove:new] 6366879630112: Downloading MPD manifest
[brightcove:new] 6366887117112: Downloading MPD manifest
[info] 6366884398112: Downloading subtitles: en
[brightcove:new] 6366888629112: Downloading MPD manifest
[brightcove:new] 6366877602112: Downloading MPD manifest
[brightcove:new] 6366880411112: Downloading MPD manifest
[brightcove:new] 6366880559112: Downloading m3u8 information
[brightcove:new] 6366875935112: Downloading MPD manifest
[brightcove:new] Extracting URL: https://players.brightcove.net/5629323166001/G9w6yOa2YM_default/index.html?videoId=6366887292112#...nd-the-hype%2F%22%7D
[generic] Extracting URL: https://www.ces.tech/videos/2025/january/ai-in-hollywood-applied-ai-the-good-the-bad-and-the-unknown/
[generic] ai-in-hollywood-applied-ai-the-good-the-bad-and-the-unknown: Downloading webpage
[brightcove:new] 6366880559112: Downloading m3u8 information
[brightcove:new] 6366880411112: Downloading MPD manifest
[generic] Extracting URL: https://www.ces.tech/videos/2025

[generic] do-you-need-a-chief-cybersecurity-officer: Downloading webpage
[brightcove:new] 6366887292112: Downloading JSON metadata
[brightcove:new] 6366887117112: Downloading MPD manifest
[brightcove:new] 6366880559112: Downloading m3u8 information
[info] 6366875935112: Downloading subtitles: en
[brightcove:new] 6366877602112: Downloading MPD manifest
[info] 6366888629112: Downloading subtitles: en
[brightcove:new] 6366880411112: Downloading MPD manifest
[brightcove:new] 6366879630112: Downloading MPD manifest
[brightcove:new] 6366881659112: Downloading JSON metadata
[brightcove:new] Extracting URL: https://players.brightcove.net/5629323166001/G9w6yOa2YM_default/index.html?videoId=6366885573112#...er-behavior%2F%22%7D
[info] 6366877602112: Downloading subtitles: en
[brightcove:new] 6366887117112: Downloading MPD manifest
[brightcove:new] 6366887292112: Downloading m3u8 information
[info] 6366880411112: Downloading subtitles: en
[brightcove:new] 6366880559112: Downloading MPD manifest
[

[generic] safe-money-venture-leaders-leading-the-way-in-cybersecurity: Downloading webpage
[generic] Extracting URL: https://www.ces.tech/videos/2025/january/data-breach-protection-and-prevention/
[brightcove:new] 6366887292112: Downloading m3u8 information
[generic] ai-in-hollywood-applied-ai-the-good-the-bad-and-the-unknown: Extracting information
[info] 6366879630112: Downloading subtitles: en
[info] 6366887117112: Downloading subtitles: en
[generic] Extracting URL: https://www.ces.tech/videos/2025/january/securing-the-iot-what-is-the-lay-of-the-land/
[brightcove:new] 6366885573112: Downloading JSON metadata
[brightcove:new] 6366880559112: Downloading MPD manifest
[generic] Extracting URL: https://www.ces.tech/videos/2025/january/the-cybersecurity-talent-gap/
[brightcove:new] 6366881659112: Downloading m3u8 information
[generic] data-breach-protection-and-prevention: Downloading webpage
[brightcove:new] 6366880559112: Downloading MPD manifest
[generic] securing-the-iot-what-is-the-l

[generic] the-cybersecurity-talent-gap: Downloading webpage
[brightcove:new] 6366887292112: Downloading m3u8 information
[generic] data-collection-privacy-and-why-you-should-care: Downloading webpage
[generic] us-cyber-trust-mark-getting-ready-for-certification: Downloading webpage
[generic] do-you-need-a-chief-cybersecurity-officer: Extracting information
[brightcove:new] Extracting URL: https://players.brightcove.net/5629323166001/G9w6yOa2YM_default/index.html?videoId=6366886773112#...the-unknown%2F%22%7D
[brightcove:new] 6366885573112: Downloading m3u8 information
[brightcove:new] 6366881659112: Downloading m3u8 information
[brightcove:new] 6366880559112: Downloading MPD manifest
[brightcove:new] 6366887292112: Downloading m3u8 information
[brightcove:new] Extracting URL: https://players.brightcove.net/5629323166001/G9w6yOa2YM_default/index.html?videoId=6366884398112#...ity-officer%2F%22%7D


[brightcove:new] 6366886773112: Downloading JSON metadata
[brightcove:new] 6366884398112: Downloading JSON metadata
[generic] safe-money-venture-leaders-leading-the-way-in-cybersecurity: Extracting information
[generic] data-collection-privacy-and-why-you-should-care: Extracting information
[generic] data-breach-protection-and-prevention: Extracting information
[brightcove:new] 6366885573112: Downloading m3u8 information
[brightcove:new] 6366881659112: Downloading m3u8 information
[info] 6366880559112: Downloading subtitles: en
[brightcove:new] 6366887292112: Downloading MPD manifest
[generic] securing-the-iot-what-is-the-lay-of-the-land: Extracting information
[generic] us-cyber-trust-mark-getting-ready-for-certification: Extracting information
[generic] the-cybersecurity-talent-gap: Extracting information
[brightcove:new] 6366881659112: Downloading m3u8 information
[brightcove:new] 6366884398112: Downloading m3u8 information
[brightcove:new] Extracting URL: https://players.brightcove